### The Radial Wavefunction Solutions
https://phys.libretexts.org/Bookshelves/Quantum_Mechanics/Quantum_Mechanics_III_(Chong)/04%3A_Identical_Particles/4.03%3A_Second_Quantization

https://quantummechanics.ucsd.edu/ph130a/130_notes/node233.html

https://galileo.phys.virginia.edu/classes/751.mf1i.fall02/HydrogenAtom.htm (no lo he usado)

https://faculty.washington.edu/seattle/physics227/reading/reading-26-27.pdf

https://www.researchgate.net/figure/Result-of-the-function-of-radial-wave-of-a-hydrogen-atom-for-i-i-4-and-5_tbl1_332921781

https://arxiv.org/pdf/2008.02946.pdf (no lo he usado)


### Polinomios de Laguerre

https://faculty.washington.edu/seattle/physics227/reading/reading-26-27.pdf

In [2]:
import sympy as sp

x = sp.Symbol('x')
r = sp.Symbol('r')

# Página 338
def laguerre(j, x):
    f = sp.exp(-x) * x**j
    for n in range(j):
        f = sp.diff(f, x)

    f = sp.exp(x) * f

    globals()[f'L_{j}'] = f

    return sp.simplify(globals()[f'L_{j}'])

# Página 339
def laguerre_associated(n, l, Z):
    j = n-l-1
    k = 2*l+1

    f = laguerre(j+k, x)

    for i in range(k):
        f = sp.diff(f, x)

    globals()[f'L_{j}^{k}'] = (-1)**k * f
    return globals()[f'L_{j}^{k}'].subs(x, 2*r*Z/n)

In [3]:
# Página 347
def radial_wavefunction(n, l, Z):
    """
    Calcula la función de onda radial para un estado dado (n, l) en un átomo con número atómico Z.
    
    Parámetros:
        n (int): Número cuántico principal.
        l (int): Número cuántico azimutal.
        Z (float): Número atómico (número de protones en el núcleo).
    
    Retorna:
        La función de onda radial.
    """
    
    # Polinomio de Laguerre generalizado
    laguerre_poly = laguerre_associated(n, l, Z)
    
    # Factor de normalización
    normalization_factor = sp.sqrt((2*Z/n)**3 * sp.factorial(n-l-1) / (2*n*sp.factorial(n+l)**3))
    
    # Exponencial
    exponential = sp.exp(-Z*r/n)
    
    # Función de onda radial
    radial_wavefunction = normalization_factor * exponential * (2*r*Z/n)**l * laguerre_poly
    
    return radial_wavefunction

In [4]:
n_max = 10
Z = 1

for n in range(1, n_max+1):
    for l in range(n):
        globals()[f'u_{n}{l}'] = r * radial_wavefunction(n, l, Z)
        globals()[f'dd_u_{n}{l}'] = sp.diff(globals()[f'u_{n}{l}'], r, r)

In [127]:
alpha = 0

n_max = 5
l = 0

h_pq = []

for i in range(1, n_max+1):
    h_pq.append([])
    for j in range(1, n_max+1):
        globals()[f'h_{i}{j}'] = -1/2 * sp.integrate(globals()[f'u_{i}0']*globals()[f'dd_u_{j}0'], (r, 0, sp.oo)) \
                                 - sp.integrate(globals()[f'u_{i}0']*globals()[f'u_{j}0'] * Z*sp.exp(-alpha*r)/r, (r, 0, sp.oo))
        h_pq[i-1].append(globals()[f'h_{i}{j}'])

In [128]:
h_pq_matrix = sp.Matrix(h_pq)
display(h_pq_matrix)

Matrix([
[                 -0.5,                     0,  5.55111512312578e-17, 2.77555756156289e-17,  -9.0205620750794e-17],
[ 5.55111512312578e-17,                -0.125, -6.10622663543836e-16,  7.7715611723761e-16, -3.88578058618805e-16],
[                    0, -3.88578058618805e-16,   -0.0555555555555558, 1.16573417585641e-15, -5.10702591327572e-15],
[-1.66533453693773e-16,  5.55111512312578e-17, -2.72004641033163e-15,             -0.03125,  1.19904086659517e-14],
[ 2.77555756156289e-17,  8.88178419700125e-16,  1.99840144432528e-15,  1.4210854715202e-14,   -0.0200000000000253]])

### The electronic energy Hamiltonian

https://qiskit-community.github.io/qiskit-nature/stubs/qiskit_nature.second_q.hamiltonians.ElectronicEnergy.html

In [17]:
import numpy as np
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy

size = len(h_pq)
h1_a = np.array(h_pq, dtype=float)
h2_aa = np.zeros((size, size, size, size))
hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa)

https://qiskit-community.github.io/qiskit-nature/howtos/vqe_ucc.html

In [69]:
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

num_spatial_orbitals = int(fermionic_op.num_spin_orbitals/2)
# The tuple of the number of alpha- and beta-spin particles
num_particles = (1, 0)

ansatz = UCCSD(
    num_spatial_orbitals,
    num_particles,
    mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals,
        num_particles,
        mapper,
    ),
)

"""
Muy importante.

Now comes the key step: choosing the initial point. Since we picked the HartreeFock initial state
before, in order to ensure we start from that, we need to initialize our initial_point with all-zero
parameters.

https://qiskit-community.github.io/qiskit-nature/howtos/vqe_ucc.html
"""

vqe_solver = VQE(Estimator(options={'shots': 2**20}), ansatz, SLSQP())
for i in range(10):
    vqe_solver.initial_point = np.zeros(ansatz.num_parameters)
    result_vqe = vqe_solver.compute_minimum_eigenvalue(operator=qubit_op)
    print(result_vqe.eigenvalue)

-0.500000244873663
-0.5000000000014178
-0.5000000019776109
-0.4999999991510124
-0.500000039862419
-0.5000001397916265
-0.5000001768398277
-0.5000002152066866
-0.5000000003129534
-0.5000000226401884


Con un número elevado de estados obtengo diferentes medidas para un mismo hamiltoniano

-0.12499999999947112

-0.12499999991149256

-0.49999990890094564

-0.4999999698305702

-0.12499999991211945

-0.12499999856306113

-0.49999993348841093

-0.12499999939637904

-0.4999999901059175

-0.4999999675014543

Puede que se arregle metiendo más shots

### Diferencia entre los diferentes Estimators

https://quantumcomputing.stackexchange.com/questions/32667/what-are-the-differences-between-the-two-estimator-in-the-qiskit

In [133]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

num_spatial_orbitals = int(fermionic_op.num_spin_orbitals/2)
# The tuple of the number of alpha- and beta-spin particles
num_particles = (1, 0)

ansatz = UCCSD(
    num_spatial_orbitals,
    num_particles,
    mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals,
        num_particles,
        mapper,
    ),
)

"""
Now comes the key step: choosing the initial point. Since we picked the HartreeFock initial state
before, in order to ensure we start from that, we need to initialize our initial_point with all-zero
parameters.

https://qiskit-community.github.io/qiskit-nature/howtos/vqe_ucc.html

Porque aquí si no lo uso no falla?
"""

seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 2**12},
    transpile_options={"seed_transpiler": seed},
)

vqe_solver = VQE(noiseless_estimator, ansatz, SPSA(maxiter=100))

for i in range(10):
    result_vqe = vqe_solver.compute_minimum_eigenvalue(operator=qubit_op)
    print(result_vqe.eigenvalue)

-0.5000000000000001
-0.5000000000000001
-0.4044426812065973
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.125


In [113]:
result_vqe.aux_operators_evaluated